## OpenStreetMap Project Data Wrangling with MongoDB
###### Yuki Steineman



### About map data

- Area: south lake tahoe, california 
- Download Link: https://mapzen.com/data/metro-extracts/
- Size: 120.2MB


### Data Wrangling

- Input file: south-lake-tahoe_california.osm (120MB)
- Output file: south-lake-tahoe_california.osm.json (25MB)

Features

1. Standarize street name (St -> Street etc).
2. Exclude k attribute which contains symbols by regex.
3. Output k attribute as "point_info".
4. Exclude some k attribute which seems not informational, such as created_by, source, and highway.

Stuck on running mongoDB

- db.tahoe.insert_many(data) function returned "ServerSelectionTimeoutError: localhost:27017: [Errno 61] Connection refused"
- I already installed pymongo, but haven't installed mongodb itself. I referred this instruction.  (https://docs.mongodb.com/manual/tutorial/install-mongodb-on-os-x/)
- To install mongodb, I used virtualenv and virtualenvwrapper. (http://docs.python-guide.org/en/latest/dev/virtualenvs/)
```
Start work on env: 
$ source /usr/local/bin/virtualenvwrapper.sh
$ workon something
Finish:
$ deactivate
```        
  

In [4]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
import xml.etree.cElementTree as ET
import pprint
import re
import codecs
import json

# regex
lower = re.compile(r'^([a-z]|_)*$')
lower_colon = re.compile(r'^([a-z]|_)*:([a-z]|_)*$')
problemchars = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')
street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)

CREATED = [ "version", "changeset", "timestamp", "user", "uid"]
EXCLUDE = ["created_by", "source" , "highway", "import_uuid", "ele", "is_in", "attribution", "ref", "exit_to"]
STREET = ["Street", "Avenue", "Boulevard", "Drive", "Court", "Place", "Square", "Lane", "Road", 
            "Trail", "Parkway", "Commons"]

# Street mapping
mapping = { "St": "Street",
            "St.": "Street",
            "Ave": "Avenue",
            "Rd.": "Road",
            "BLVD": "Boulevard",
            "DR": "Drive",
            "CT": "Court",
            "PL": "Place",
            "SQ": "Square",
            "LN": "Lane",
            "TRL": "Trail",
            "PKWY": "Parkway",
            "CMNS": "Commons"
          }

def process_map(file_in, pretty = False):
    # You do not need to change this file
    file_out = "{0}.json".format(file_in)
    data = []
    with codecs.open(file_out, "w") as fo:
        fo.write("[")
        for _, element in ET.iterparse(file_in):
            el = shape_element(element)
            if el:
                data.append(el)
                if pretty:
                    fo.write(json.dumps(el, indent=2)+",\n")
                else:
                    fo.write(json.dumps(el) + ",\n")
        fo.write("]")
    return data

In [6]:
def shape_element(element):
    node = {}
    address = {}
    node_refs = []
    info = {}
    
    if element.tag == "node" or element.tag == "way" :
        created = {}
        pos = [None, None]
        for attr in element.attrib.keys():
            if attr == 'id':
                node["id"] = element.attrib[attr]
            if attr == "visible":
                node["visible"] = element.attrib[attr]
            if attr in CREATED:
                created[attr] = element.attrib[attr]
            # pos
            if attr == 'lat':
                pos[0] = float(element.attrib[attr])
            if attr == 'lon':
                pos[1] = float(element.attrib[attr])
                
        node["type"] = element.tag
        node["created"] = created
        if pos[0]:
            node["pos"] = pos
       
        for tag in element.iter('tag'):
            k = tag.attrib['k']
            # address standarize
            if k.startswith("addr:"):
                if lower_colon.search(k):
                    v = tag.attrib['v']
                    matcher = street_type_re.search(v)
                    if matcher:
                        streetname = matcher.group(0) 
                        if streetname in mapping:
                            # set standarized street name
                            address[k[5:]] = v.replace(streetname, mapping[streetname])
                        else:
                            address[k[5:]] = v
                    else:
                        address[k[5:]] = v
                    
            # not address
            elif lower.search(k):
                # exclude unnecessary tag defined in 'EXCLUDE'
                if not k in EXCLUDE:
                    info[k] = tag.attrib['v']
                    
            # contains symbols
            elif problemchars.search(k):
                pass

        if address:
            node["address"] = address
        if info:
            node["point_info"] = info 
            
        # node_refs
        for tag in element.iter('nd'):
            ref = tag.attrib['ref']
            node_refs.append(ref)
            
        if node_refs:
            node["node_refs"] = node_refs
            
        return node
        
    else:
        return None

if __name__ == "__main__":
    data = process_map('south-lake-tahoe_california.osm', True)
    #data = process_map('test.osm', True)
    pprint.pprint(data[0])
    
    

{'created': {'changeset': '12599511',
             'timestamp': '2012-08-03T15:23:02Z',
             'uid': '169004',
             'user': 'oldtopos',
             'version': '3'},
 'id': '26798725',
 'pos': [39.4433397, -120.0117505],
 'type': 'node'}


##### Set up MongoDB

In [1]:
def get_db():
    # For local use
    from pymongo import MongoClient
    client = MongoClient('localhost:27017')
    # 'examples' here is the database name. It will be created if it does not exist.
    db = client.laketahoe
    return db

db = get_db()
print db

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), u'laketahoe')


##### Read osm (json format) file

In [21]:
import json
with open("south-lake-tahoe_california.osm.json", "r") as fi:
# remove comma
    data = json.load(fi)
pprint.pprint(data[0])

{u'created': {u'changeset': u'12599511',
              u'timestamp': u'2012-08-03T15:23:02Z',
              u'uid': u'169004',
              u'user': u'oldtopos',
              u'version': u'3'},
 u'id': u'26798725',
 u'pos': [39.4433397, -120.0117505],
 u'type': u'node'}


##### Insert to database

In [22]:
db.tahoe.insert_many(data)
# not insert()

In [23]:
db.tahoe.find_one()

{u'_id': ObjectId('5732cf73ce0450d2cf2dfe00'),
 u'created': {u'changeset': u'12599511',
  u'timestamp': u'2012-08-03T15:23:02Z',
  u'uid': u'169004',
  u'user': u'oldtopos',
  u'version': u'3'},
 u'id': u'26798725',
 u'pos': [39.4433397, -120.0117505],
 u'type': u'node'}

### Explore dataset

##### Number of contribution

Total 98,847 posts 

In [47]:
tcount = db.tahoe.find({"created.user": { "$ne": None }}).count() 
print 'Number of posts:', tcount

Number of posts: 724873


##### User contribution ranking
Most posted user is 'woodpeck_fixbot', 17% of the total.

In [33]:
import math
def make_pipeline():
    # complete the aggregation pipeline
    pipeline = [
         { "$match": {"created.user": { "$ne": None }}}, 
         {"$group": {"_id" : "$created.user", "count" : {"$sum": 1} }},
         {"$sort" : { "count" : -1 } },
         {"$limit" : 10 }]
    return pipeline

def aggregate(db, pipeline):
    return [doc for doc in db.tahoe.aggregate(pipeline)]

pipeline = make_pipeline()
result = aggregate(db, pipeline)

for r in result:
    r['ratio'] = math.floor((float(r['count']) / float(tcount)) * 100)
    print 'User: {0}, Count: {1}, Ratio: {2}'.format(r['_id'], r['count'], r['ratio'])


'User: woodpeck_fixbot, Count: 127156, Ratio: 17.0'
'User: nmixter, Count: 103801, Ratio: 14.0'
'User: theangrytomato, Count: 82896, Ratio: 11.0'
'User: AndrewBuck, Count: 66328, Ratio: 9.0'
'User: MelanieOriet, Count: 26138, Ratio: 3.0'
'User: lucaswoj, Count: 22137, Ratio: 3.0'
'User: paulmach, Count: 21365, Ratio: 2.0'
'User: abschiff, Count: 15213, Ratio: 2.0'
'User: wallclimber21, Count: 14867, Ratio: 2.0'
'User: Marshy8, Count: 14016, Ratio: 1.0'


##### Most contributed year 

- 2012: 25% (of the total)
- 2009: 23%
- 2015: 15%
- 2014: 15%
- 2013: 10%

In [55]:
def make_pipeline():
    # complete the aggregation pipeline
    pipeline = [
         { "$match": {"created.timestamp": { "$ne": None }}}, 
         {"$group": {"_id" : {"$substr": [ "$created.timestamp", 0, 4 ]}, "count" : {"$sum": 1} }},
         {"$sort" : { "count" : -1 } }
        ]
    return pipeline

def aggregate(db, pipeline):
    return [doc for doc in db.tahoe.aggregate(pipeline)]

pipeline = make_pipeline()
result = aggregate(db, pipeline)

for r in result:
    r['ratio'] = math.floor((float(r['count']) / float(tcount)) * 100)
    print 'Year: {0}, Count: {1}, Ratio: {2}'.format(r['_id'],r['count'],r['ratio'])
#pprint.pprint(result)

Year: 2012, Count: 182291, Ratio: 25.0
Year: 2009, Count: 167322, Ratio: 23.0
Year: 2015, Count: 112548, Ratio: 15.0
Year: 2014, Count: 110582, Ratio: 15.0
Year: 2013, Count: 79033, Ratio: 10.0
Year: 2016, Count: 26409, Ratio: 3.0
Year: 2010, Count: 20176, Ratio: 2.0
Year: 2011, Count: 19110, Ratio: 2.0
Year: 2008, Count: 5811, Ratio: 0.0
Year: 2007, Count: 1591, Ratio: 0.0


##### Detail of contribution by user and year
- User 'woodpeck_fixbot' contributed most but only active in 2009.
- Two users 'nmixter' and 'AndrewBuck' were active in 2012.
- User 'theangrytomato' was active in 2014 and 2015.

In [53]:
def make_pipeline():
    # complete the aggregation pipeline
    pipeline = [
         { "$match": {"created.user": { "$ne": None }}}, 
         { "$match": {"created.timestamp": { "$ne": None }}}, 
         {"$group": {"_id" : {"user":"$created.user", "year":{"$substr": [ "$created.timestamp", 0, 4 ]}}, "count" : {"$sum": 1} }},
         {"$match": {"count": { "$gt": 10000 }}},
         {"$sort" : { "count" : -1} }
        ]
    return pipeline

def aggregate(db, pipeline):
    return [doc for doc in db.tahoe.aggregate(pipeline)]

pipeline = make_pipeline()
result = aggregate(db, pipeline)

for r in result:
    r['ratio'] = math.floor((float(r['count']) / float(tcount)) * 100)
    print 'User: {0}, Year: {1}, Count: {2}, Ratio: {3}'.format(r['_id']['user'],r['_id']['year'],r['count'],r['ratio'])
#pprint.pprint(result)

User: woodpeck_fixbot, Year: 2009, Count: 127156, Ratio: 17.0
User: nmixter, Year: 2012, Count: 86458, Ratio: 11.0
User: AndrewBuck, Year: 2012, Count: 51902, Ratio: 7.0
User: theangrytomato, Year: 2015, Count: 41349, Ratio: 5.0
User: theangrytomato, Year: 2014, Count: 30271, Ratio: 4.0
User: lucaswoj, Year: 2014, Count: 22136, Ratio: 3.0
User: MelanieOriet, Year: 2013, Count: 20406, Ratio: 2.0
User: AndrewBuck, Year: 2013, Count: 14426, Ratio: 1.0
User: Eureka gold, Year: 2012, Count: 12762, Ratio: 1.0
User: paulmach, Year: 2015, Count: 11951, Ratio: 1.0
User: abschiff, Year: 2009, Count: 11128, Ratio: 1.0
User: Marshy8, Year: 2015, Count: 10229, Ratio: 1.0


##### Detail of amenity
Parking is the most common amenity. Next is school.

In [36]:
def make_pipeline():
    # complete the aggregation pipeline
    pipeline = [
         {"$match": {"point_info.amenity": { "$ne": None }}}, 
         {"$unwind": "$point_info"},
         {"$group": {"_id" : "$point_info.amenity", "count" : {"$sum": 1} }},
         {"$sort" : { "count" : -1 } },
         {"$limit" : 15 }
        ]
    return pipeline

def aggregate(db, pipeline):
    return [doc for doc in db.tahoe.aggregate(pipeline)]

pipeline = make_pipeline()
result = aggregate(db, pipeline)

for r in result:
    print 'Amenity: {0}, Count: {1}'.format(r['_id'],r['count'])
# pprint.pprint(result)

Amenity: parking, Count: 371
Amenity: school, Count: 158
Amenity: post_office, Count: 79
Amenity: toilets, Count: 79
Amenity: restaurant, Count: 65
Amenity: grave_yard, Count: 27
Amenity: fast_food, Count: 26
Amenity: hospital, Count: 25
Amenity: place_of_worship, Count: 22
Amenity: cafe, Count: 21
Amenity: fuel, Count: 19
Amenity: bank, Count: 14
Amenity: drinking_water, Count: 14
Amenity: fire_station, Count: 12
Amenity: pharmacy, Count: 10


##### What is the most eaten in Lake Tahoe
```
1. Burger
2. Mexican
3. Pizza
3. American
3. Sandwich
```

In [37]:
def make_pipeline():
    # complete the aggregation pipeline
    pipeline = [
         {"$match": {"point_info.cuisine": { "$ne": None }}}, 
         {"$unwind": "$point_info"},
         {"$group": {"_id" : "$point_info.cuisine", "count" : {"$sum": 1} }},
         {"$sort" : { "count" : -1 } }
        ]
    return pipeline

def aggregate(db, pipeline):
    return [doc for doc in db.tahoe.aggregate(pipeline)]

pipeline = make_pipeline()
result = aggregate(db, pipeline)

for r in result:
    print 'Cuisine: {0}, Count: {1}'.format(r['_id'],r['count'])

Cuisine: burger, Count: 8
Cuisine: mexican, Count: 7
Cuisine: pizza, Count: 5
Cuisine: american, Count: 5
Cuisine: sandwich, Count: 5
Cuisine: coffee_shop, Count: 4
Cuisine: chicken, Count: 3
Cuisine: thai, Count: 3
Cuisine: sushi, Count: 2
Cuisine: italian, Count: 2
Cuisine: regional, Count: 2
Cuisine: german, Count: 1
Cuisine: Hawaiian, Count: 1
Cuisine: chinese, Count: 1
Cuisine: asian, Count: 1


#### Some trick I learned from above

- Substring; pick up year from timestamp
```
"year":{"$substr": [ "$created.timestamp", 0, 4 ]}}
```

- Group with Nested element
```
 {"$unwind": "$point_info"},
 {"$group": {"_id" : "$point_info.amenity", "count" : {"$sum": 1} }}
```

- Multiple sort key
```
{"$sort" : { "year" : -1, "count" : -1} }
```

### Conclusion

In "Detail of contribution by user and year" section, I planned to get the most contributed user in each year. But I couldn't figure out the way to sort and limit in each year.  
MongoDB is still not handy for me, however, we don't need to transform json format to save in database, this feature is quite attractive. 